## Anothetr example pydantic and fastAPI

### Code

In [ ]:
# Install required packages
!pip install fastapi pydantic uvicorn nest-asyncio

import nest_asyncio
nest_asyncio.apply()  # Allow running FastAPI in Jupyter

from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from enum import Enum
from datetime import datetime
import uvicorn
from IPython.display import Markdown, display
import uuid

# Define enums for product categories
class ProductCategory(str, Enum):
    ELECTRONICS = "electronics"
    CLOTHING = "clothing"
    BOOKS = "books"
    HOME = "home"
    BEAUTY = "beauty"
    FOOD = "food"

# Request Pydantic Model for product creation
class ProductCreateRequest(BaseModel):
    name: str = Field(..., min_length=3, max_length=100, description="Product name")
    description: str = Field(..., min_length=10, max_length=1000, description="Product description")
    price: float = Field(..., gt=0, description="Product price in USD")
    category: ProductCategory = Field(..., description="Product category")
    in_stock: bool = Field(True, description="Whether the product is in stock")
    tags: List[str] = Field(default=[], description="Product tags for search and categorization")
    
    @validator('price')
    def price_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Price must be positive')
        return round(v, 2)  # Round to 2 decimal places
    
    class Config:
        json_schema_extra = {
            "example": {
                "name": "Wireless Headphones",
                "description": "Premium noise-canceling wireless headphones with 20-hour battery life",
                "price": 149.99,
                "category": "electronics",
                "in_stock": True,
                "tags": ["audio", "wireless", "headphones"]
            }
        }

# Response Pydantic Model for products
class ProductResponse(BaseModel):
    id: str
    name: str
    description: str
    price: float
    category: ProductCategory
    in_stock: bool
    tags: List[str]
    created_at: datetime
    
    class Config:
        json_schema_extra = {
            "example": {
                "id": "123e4567-e89b-12d3-a456-426614174000",
                "name": "Wireless Headphones",
                "description": "Premium noise-canceling wireless headphones with 20-hour battery life",
                "price": 149.99,
                "category": "electronics",
                "in_stock": True,
                "tags": ["audio", "wireless", "headphones"],
                "created_at": "2025-03-18T12:00:00Z"
            }
        }

# Query Parameters Model for product filtering
class ProductFilterParams(BaseModel):
    category: Optional[ProductCategory] = None
    min_price: Optional[float] = Field(None, ge=0)
    max_price: Optional[float] = Field(None, gt=0)
    in_stock_only: bool = False
    
    @validator('max_price')
    def max_price_greater_than_min_price(cls, v, values):
        if v is not None and 'min_price' in values and values['min_price'] is not None:
            if v <= values['min_price']:
                raise ValueError('max_price must be greater than min_price')
        return v

# Mock database
products_db = {}

# Create FastAPI application
app = FastAPI(title="Product API")

# Create product endpoint
@app.post("/products/", response_model=ProductResponse, status_code=201)
async def create_product(product: ProductCreateRequest):
    """
    Create a new product with the provided details
    """
    product_id = str(uuid.uuid4())
    product_data = product.model_dump()
    product_response = {
        "id": product_id,
        "created_at": datetime.now(),
        **product_data
    }
    products_db[product_id] = product_response
    return product_response

# Get products endpoint with query filtering
@app.get("/products/", response_model=List[ProductResponse])
async def get_products(
    category: Optional[ProductCategory] = None,
    min_price: Optional[float] = Query(None, ge=0),
    max_price: Optional[float] = Query(None, gt=0),
    in_stock_only: bool = False
):
    """
    Get products with optional filtering
    """
    # Create filter params and validate them
    filter_params = ProductFilterParams(
        category=category,
        min_price=min_price,
        max_price=max_price,
        in_stock_only=in_stock_only
    )
    
    # Apply filters
    filtered_products = []
    for product in products_db.values():
        # Filter by category
        if filter_params.category and product["category"] != filter_params.category:
            continue
            
        # Filter by min price
        if filter_params.min_price is not None and product["price"] < filter_params.min_price:
            continue
            
        # Filter by max price
        if filter_params.max_price is not None and product["price"] > filter_params.max_price:
            continue
            
        # Filter by stock
        if filter_params.in_stock_only and not product["in_stock"]:
            continue
            
        filtered_products.append(product)
        
    return filtered_products

# Get product by ID endpoint
@app.get("/products/{product_id}", response_model=ProductResponse)
async def get_product(product_id: str):
    """
    Get a specific product by its ID
    """
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    return products_db[product_id]

# Test the models in Jupyter
def test_pydantic_models():
    display(Markdown("## Testing Product API Pydantic Models"))
    
    # Test valid product creation
    try:
        product = ProductCreateRequest(
            name="Ergonomic Desk Chair",
            description="Adjustable office chair with lumbar support and breathable mesh back",
            price=199.99,
            category=ProductCategory.HOME,
            tags=["furniture", "office", "ergonomic"]
        )
        print("✅ Valid product created:", product.model_dump())
    except Exception as e:
        print("❌ Error creating valid product:", e)
    
    # Test invalid product (negative price)
    try:
        product = ProductCreateRequest(
            name="Invalid Product",
            description="This product has an invalid price",
            price=-10.99,
            category=ProductCategory.ELECTRONICS
        )
        print("Product:", product.model_dump())
    except Exception as e:
        print("❌ Validation error (expected):", e)
    
    # Test filter parameters
    try:
        filters = ProductFilterParams(
            category=ProductCategory.ELECTRONICS,
            min_price=50,
            max_price=200,
            in_stock_only=True
        )
        print("✅ Valid filters created:", filters.model_dump())
    except Exception as e:
        print("❌ Error creating filters:", e)
    
    # Test invalid filter parameters
    try:
        filters = ProductFilterParams(
            min_price=200,
            max_price=100  # max_price less than min_price
        )
        print("Filters:", filters.model_dump())
    except Exception as e:
        print("❌ Validation error (expected):", e)

# Run the test function
test_pydantic_models()

# Sample usage code
display(Markdown("""
## Example Usage in Jupyter

```python
# Add products to the database
product1 = ProductCreateRequest(
    name="Wireless Headphones",
    description="Premium noise-canceling wireless headphones with 20-hour battery life",
    price=149.99,
    category=ProductCategory.ELECTRONICS,
    tags=["audio", "wireless", "headphones"]
)
product2 = ProductCreateRequest(
    name="Cotton T-Shirt",
    description="Comfortable 100% organic cotton t-shirt, available in various colors",
    price=24.99,
    category=ProductCategory.CLOTHING,
    tags=["t-shirt", "cotton", "casual"]
)

# Use the API functions directly in Jupyter
product1_response = await create_product(product1)
product2_response = await create_product(product2)

# Get all products
all_products = await get_products()
print(f"All products: {len(all_products)}")

# Filter products
electronics = await get_products(category=ProductCategory.ELECTRONICS)
print(f"Electronics products: {len(electronics)}")

# Get specific product
product = await get_product(product1_response['id'])
print(f"Retrieved product: {product['name']}")
```

Or to run as an actual API server:

```python
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)
```

Then access: http://127.0.0.1:8000/docs in your browser to see the Swagger UI
"""))

### Test_1

In [1]:
# Install required packages
# !pip install fastapi pydantic uvicorn nest-asyncio

import nest_asyncio
nest_asyncio.apply()  # Allow running FastAPI in Jupyter

from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from enum import Enum
from datetime import datetime
import uvicorn
from IPython.display import Markdown, display
import uuid

In [ ]:
# Define enums for product categories
class ProductCategory(str, Enum):
    ELECTRONICS = "electronics"
    CLOTHING = "clothing"
    BOOKS = "books"
    HOME = "home"
    BEAUTY = "beauty"
    FOOD = "food"

In [ ]:
# Request Pydantic Model for product creation
class ProductCreateRequest(BaseModel):
    name: str = Field(..., min_length=3, max_length=100, description="Product name")
    description: str = Field(..., min_length=10, max_length=1000, description="Product description")
    price: float = Field(..., gt=0, description="Product price in USD")
    category: ProductCategory = Field(..., description="Product category")
    in_stock: bool = Field(True, description="Whether the product is in stock")
    tags: List[str] = Field(default=[], description="Product tags for search and categorization")
    
    @validator('price')
    def price_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Price must be positive')
        return round(v, 2)  # Round to 2 decimal places
    
    class Config:
        json_schema_extra = {
            "example": {
                "name": "Wireless Headphones",
                "description": "Premium noise-canceling wireless headphones with 20-hour battery life",
                "price": 149.99,
                "category": "electronics",
                "in_stock": True,
                "tags": ["audio", "wireless", "headphones"]
            }
        }

# Response Pydantic Model for products
class ProductResponse(BaseModel):
    id: str
    name: str
    description: str
    price: float
    category: ProductCategory
    in_stock: bool
    tags: List[str]
    created_at: datetime
    
    class Config:
        json_schema_extra = {
            "example": {
                "id": "123e4567-e89b-12d3-a456-426614174000",
                "name": "Wireless Headphones",
                "description": "Premium noise-canceling wireless headphones with 20-hour battery life",
                "price": 149.99,
                "category": "electronics",
                "in_stock": True,
                "tags": ["audio", "wireless", "headphones"],
                "created_at": "2025-03-18T12:00:00Z"
            }
        }

# Query Parameters Model for product filtering
class ProductFilterParams(BaseModel):
    category: Optional[ProductCategory] = None
    min_price: Optional[float] = Field(None, ge=0)
    max_price: Optional[float] = Field(None, gt=0)
    in_stock_only: bool = False
    
    @validator('max_price')
    def max_price_greater_than_min_price(cls, v, values):
        if v is not None and 'min_price' in values and values['min_price'] is not None:
            if v <= values['min_price']:
                raise ValueError('max_price must be greater than min_price')
        return v

In [ ]:
# Mock database
products_db = {}

# Create FastAPI application
app = FastAPI(title="Product API")

# Create product endpoint
@app.post("/products/", response_model=ProductResponse, status_code=201)
async def create_product(product: ProductCreateRequest):
    """
    Create a new product with the provided details
    """
    product_id = str(uuid.uuid4())
    product_data = product.model_dump()
    product_response = {
        "id": product_id,
        "created_at": datetime.now(),
        **product_data
    }
    products_db[product_id] = product_response
    return product_response

# Get products endpoint with query filtering
@app.get("/products/", response_model=List[ProductResponse])
async def get_products(
    category: Optional[ProductCategory] = None,
    min_price: Optional[float] = Query(None, ge=0),
    max_price: Optional[float] = Query(None, gt=0),
    in_stock_only: bool = False
):
    """
    Get products with optional filtering
    """
    # Create filter params and validate them
    filter_params = ProductFilterParams(
        category=category,
        min_price=min_price,
        max_price=max_price,
        in_stock_only=in_stock_only
    )
    
    # Apply filters
    filtered_products = []
    for product in products_db.values():
        # Filter by category
        if filter_params.category and product["category"] != filter_params.category:
            continue
            
        # Filter by min price
        if filter_params.min_price is not None and product["price"] < filter_params.min_price:
            continue
            
        # Filter by max price
        if filter_params.max_price is not None and product["price"] > filter_params.max_price:
            continue
            
        # Filter by stock
        if filter_params.in_stock_only and not product["in_stock"]:
            continue
            
        filtered_products.append(product)
        
    return filtered_products

# Get product by ID endpoint
@app.get("/products/{product_id}", response_model=ProductResponse)
async def get_product(product_id: str):
    """
    Get a specific product by its ID
    """
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    return products_db[product_id]

# Test the models in Jupyter
def test_pydantic_models():
    display(Markdown("## Testing Product API Pydantic Models"))
    
    # Test valid product creation
    try:
        product = ProductCreateRequest(
            name="Ergonomic Desk Chair",
            description="Adjustable office chair with lumbar support and breathable mesh back",
            price=199.99,
            category=ProductCategory.HOME,
            tags=["furniture", "office", "ergonomic"]
        )
        print("✅ Valid product created:", product.model_dump())
    except Exception as e:
        print("❌ Error creating valid product:", e)
    
    # Test invalid product (negative price)
    try:
        product = ProductCreateRequest(
            name="Invalid Product",
            description="This product has an invalid price",
            price=-10.99,
            category=ProductCategory.ELECTRONICS
        )
        print("Product:", product.model_dump())
    except Exception as e:
        print("❌ Validation error (expected):", e)
    
    # Test filter parameters
    try:
        filters = ProductFilterParams(
            category=ProductCategory.ELECTRONICS,
            min_price=50,
            max_price=200,
            in_stock_only=True
        )
        print("✅ Valid filters created:", filters.model_dump())
    except Exception as e:
        print("❌ Error creating filters:", e)
    
    # Test invalid filter parameters
    try:
        filters = ProductFilterParams(
            min_price=200,
            max_price=100  # max_price less than min_price
        )
        print("Filters:", filters.model_dump())
    except Exception as e:
        print("❌ Validation error (expected):", e)

# Run the test function
test_pydantic_models()

# Sample usage code
display(Markdown("""
## Example Usage in Jupyter

```python
# Add products to the database
product1 = ProductCreateRequest(
    name="Wireless Headphones",
    description="Premium noise-canceling wireless headphones with 20-hour battery life",
    price=149.99,
    category=ProductCategory.ELECTRONICS,
    tags=["audio", "wireless", "headphones"]
)
product2 = ProductCreateRequest(
    name="Cotton T-Shirt",
    description="Comfortable 100% organic cotton t-shirt, available in various colors",
    price=24.99,
    category=ProductCategory.CLOTHING,
    tags=["t-shirt", "cotton", "casual"]
)

# Use the API functions directly in Jupyter
product1_response = await create_product(product1)
product2_response = await create_product(product2)

# Get all products
all_products = await get_products()
print(f"All products: {len(all_products)}")

# Filter products
electronics = await get_products(category=ProductCategory.ELECTRONICS)
print(f"Electronics products: {len(electronics)}")

# Get specific product
product = await get_product(product1_response['id'])
print(f"Retrieved product: {product['name']}")
```

Or to run as an actual API server:

```python
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)
```

Then access: http://127.0.0.1:8000/docs in your browser to see the Swagger UI
"""))